In [133]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm import tqdm
import pickle
from pandarallel import pandarallel
from time import time
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import scipy.sparse as sp
from scipy.stats import linregress
from goatools.obo_parser import GODag
from goatools.mapslim import mapslim
from goatools.gosubdag.gosubdag import GoSubDag
import requests
import os
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import defaultdict
import random

from bertopic import BERTopic
from bertopic.backend import BaseEmbedder
from bertopic.cluster import BaseCluster
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired

from typing import List, Dict, Tuple, Set, Any, Optional, Union

import networkx as nx

from vocabulary_functions import get_mutated, get_parents, set_difference, set_intersection, load_tokenizers, calc_agreement, calc_dice_idx_only

In [2]:
pandarallel.initialize(nb_workers=20, progress_bar=True)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# Download GO DAG (ontology structure)
obo_url = "http://purl.obolibrary.org/obo/go/go-basic.obo"
obo_path = "go-basic.obo"

# Download OBO file if not already downloaded
if not os.path.exists(obo_path):
    with open(obo_path, "w") as f:
        f.write(requests.get(obo_url).text)

# Parse GO DAG
go_dag = GODag(obo_path)

# Download GO Slim generic terms
goslim_url = "http://current.geneontology.org/ontology/subsets/goslim_generic.obo"
goslim_path = "goslim_generic.obo"

if not os.path.exists(goslim_path):
    with open(goslim_path, "w") as f:
        f.write(requests.get(goslim_url).text)

# Parse GO Slim DAG
goslim_dag = GODag(goslim_path)
goslim_terms = set(goslim_dag.keys())

go_subdag_slim = GoSubDag(goslim_terms, go_dag)

goslim_id2name = {go_id : go_dag[go_id].name for go_id in goslim_terms}
goslim_name2id = {go_dag[go_id].name : go_id for go_id in goslim_terms}

go-basic.obo: fmt(1.2) rel(2025-03-16) 43,544 Terms
goslim_generic.obo: fmt(1.2) rel(go/2025-03-16/subsets/goslim_generic.owl) 206 Terms
 GoSubDag: 206 sources in 335 GOs rcnt(True). 65 alt GO IDs
 GoSubDag: namedtuple fields: NS level depth GO alt GO_name dcnt D1 id
 GoSubDag: relationships: set()


In [4]:
# 'dataset': {'uniref50', 'uniref90'}
# 'is_pretokenizer': {True, False}
# 'subs_matrix': {'blosum45', 'blosum62', 'pam70', 'pam250'}
# 'mutation_cutoff': {0.7, 0.8, 0.9}
# 'min_mutation_freq': {0, 0.05,. 0.005}
# 'min_mutation_len': {3}
# 'max_mutation_len': {12}
# 'vocab_size': list=[800, 1600, 3200, 6400, 12800, 25600, 51200]

vocab_sizes = [800, 1600, 3200, 6400, 12800, 25600]
vocab_sizes = [6400, 12800, 25600]
uniref_id = "50"

tokenizer_opts_list = [
    {
        'is_mut': True,
        'dataset': f'uniref{uniref_id}',
        'is_pretokenizer': False,
        'subs_matrix': 'blosum62',
        'mutation_cutoff': 0.7,
        'min_mutation_freq': 0.05,
        'min_mutation_len': 3,
        'max_mutation_len': 12,
        'vocab_size': vocab_sizes
    },
    # {
    #     'is_mut': True,
    #     'dataset': f'uniref{uniref_id}',
    #     'is_pretokenizer': False,
    #     'subs_matrix': 'pam70',
    #     'mutation_cutoff': 0.7,
    #     'min_mutation_freq': 0.05,
    #     'min_mutation_len': 3,
    #     'max_mutation_len': 12,
    #     'vocab_size': vocab_sizes
    # },
    # {
    #     'is_mut': True,
    #     'dataset': f'uniref{uniref_id}',
    #     'is_pretokenizer': True,
    #     'subs_matrix': 'blosum62',
    #     'mutation_cutoff': 0.7,
    #     'min_mutation_freq': 0.05,
    #     'min_mutation_len': 3,
    #     'max_mutation_len': 12,
    #     'vocab_size': vocab_sizes
    # },
    {
        'is_mut': True,
        'dataset': f'uniref{uniref_id}',
        'is_pretokenizer': True,
        'subs_matrix': 'pam70',
        'mutation_cutoff': 0.7,
        'min_mutation_freq': 0.05,
        'min_mutation_len': 3,
        'max_mutation_len': 12,
        'vocab_size': vocab_sizes
    },
]

In [5]:
tokenizer_list = load_tokenizers(tokenizer_opts_list, 'hf')
inner_vocab_list = load_tokenizers(tokenizer_opts_list, 'vocab')

vocab_list = {}
for name, tokenizer in tokenizer_list.items():
    vocab_list[name] = list(set([token for token, idx in tokenizer.get_vocab().items()]))

In [6]:
methods = [method_name[:-len(str(vocab_sizes[0]))-1] for method_name in list(tokenizer_list.keys())[::len(vocab_sizes)]]
methods2names = {mn:mn.replace('mut', 'evo').replace('std', '').replace('blosum', 'BLOSUM').replace('pam', 'PAM').replace('pre', 'Pre') for mn in methods}
methods2names = {k: ' '.join(v.split()[:-2]) if 'evoBPE' in v else v for k, v in methods2names.items()}
methods2names

{'PUMA blosum62 0.7 0.05': 'PUMA BLOSUM62 0.7 0.05',
 'PUMA pre pam70 0.7 0.05': 'PUMA Pre PAM70 0.7 0.05'}

In [7]:
vocab_lineage_list = {}
for k, v in inner_vocab_list.items():
    template_dict = {
        
    }
    vocab_lineage_list[k] = {token:{
                                'frequency': -1,
                                'order': -1,
                                'parent_pair': [],
                                'parent_mutation': "",
                                'parent_mutation_similarity': -1,
                                'partner_pair_self': False,
                                'partner_pair_left': [],
                                'partner_pair_right': [],
                                'child_pair': [],
                                'child_mutation': []
                            } for token in v.keys()}

for method_name, vocab in tqdm(inner_vocab_list.items()):
    for token, inner_vocab_elements in vocab.items():
        vocab_lineage_list[method_name][token]['frequency'] = inner_vocab_elements['frequency']
        vocab_lineage_list[method_name][token]['order'] = inner_vocab_elements['order']
        vocab_lineage_list[method_name][token]['parent_pair'] = inner_vocab_elements['pair'] if 'pair' in inner_vocab_elements else []
        vocab_lineage_list[method_name][token]['parent_mutation'] = inner_vocab_elements['parent'] if 'parent' in inner_vocab_elements else ""
        vocab_lineage_list[method_name][token]['parent_mutation_similarity'] = inner_vocab_elements['similarity'] if 'similarity' in inner_vocab_elements else -1

        if 'pair' in inner_vocab_elements:
            if inner_vocab_elements['pair'][0] == inner_vocab_elements['pair'][1]:
                vocab_lineage_list[method_name][inner_vocab_elements['pair'][0]]['partner_pair_self'] = True
                vocab_lineage_list[method_name][inner_vocab_elements['pair'][0]]['child_pair'].append(token)
            else:
                vocab_lineage_list[method_name][inner_vocab_elements['pair'][0]]['partner_pair_right'].append(inner_vocab_elements['pair'][1])
                vocab_lineage_list[method_name][inner_vocab_elements['pair'][1]]['partner_pair_left'].append(inner_vocab_elements['pair'][0])
                vocab_lineage_list[method_name][inner_vocab_elements['pair'][0]]['child_pair'].append(token)
                vocab_lineage_list[method_name][inner_vocab_elements['pair'][1]]['child_pair'].append(token)
        if 'parent' in inner_vocab_elements:
                vocab_lineage_list[method_name][inner_vocab_elements['parent']]['child_mutation'].append(token)

100%|██████████| 6/6 [00:00<00:00, 47.65it/s]


In [168]:
# Connect to DB
db_file = "/cta/share/users/uniprot/human/human.db"
conn = sqlite3.connect(db_file)

df_protein = pd.read_sql(f"""SELECT Entry as uniprot_id, Sequence as sequence
                          FROM proteins
                          WHERE Entry IN (SELECT uniprot_accession FROM uniref{uniref_id}_distilled)""", conn)
df_protein = df_protein[df_protein['sequence'].str.len() < 3000].reset_index(drop=True)

df_protein_sliced = pd.read_sql(f"SELECT uniprot_id, sequence FROM uniref{uniref_id}_domain_sliced_plddt70", conn)
df_protein_sliced = df_protein_sliced[df_protein_sliced['uniprot_id'].isin(df_protein['uniprot_id'])].reset_index(drop=True)

df_quickgo = pd.read_sql(f"SELECT uniprot_id, go_id, go_evidence FROM uniprot_quickgo_annotations", conn)
df_quickgo = df_quickgo[df_quickgo['uniprot_id'].isin(df_protein['uniprot_id'])]

# Filter by evidence code
go_evidence_experimental = ['EXP', 'IDA', 'IPI', 'IMP', 'IGI', 'IEP']
go_evidence_computational = ['ISS', 'ISO', 'ISA', 'ISM', 'RCA', 'TAS', 'IEA']
df_quickgo = df_quickgo[df_quickgo['go_evidence'].isin(go_evidence_experimental+go_evidence_computational)].reset_index(drop=True)

# Convert go_ids to go_slim_ids
df_quickgo['go_slim_id'] = df_quickgo['go_id'].apply(lambda x: list(mapslim(x, go_dag, goslim_dag)[0]) if x in go_dag else [])
df_quickgo['go_slim_id'] = df_quickgo['go_slim_id'].apply(lambda x: x[0] if len(x) > 0 else '')
df_quickgo = df_quickgo[df_quickgo['go_slim_id'] != '']

# Drop duplicates
# df_quickgo = df_quickgo.drop_duplicates(subset=['uniprot_id', 'go_slim_id']).drop(columns=['go_id', 'go_evidence']).rename(columns={'go_slim_id': 'go_id'}).reset_index(drop=True)
df_quickgo = df_quickgo.groupby(['uniprot_id', 'go_slim_id']).count().reset_index().sort_values(['uniprot_id', 'go_id'], ascending=False)
df_quickgo = df_quickgo.drop(columns=['go_id', 'go_evidence']).rename(columns={'go_slim_id': 'go_id'}).reset_index(drop=True)

# # Drop infrequent go terms
df_quickgo = df_quickgo[df_quickgo['go_id'].isin(df_quickgo['go_id'].value_counts()[df_quickgo['go_id'].value_counts()>20].index)].reset_index(drop=True)

conn.close()

In [169]:
df_quickgo['go_name'] = df_quickgo['go_id'].apply(lambda go_id: go_dag[go_id].name)
df_quickgo['go_aspect'] = df_quickgo['go_id'].apply(lambda go_id: go_dag[go_id].namespace)
df_quickgo

,uniprot_id,go_id,go_name,go_aspect
0,X6RLR1,GO:0000910,cytokinesis,biological_process
1,X6RLR1,GO:0005730,nucleolus,cellular_component
2,X6RLR1,GO:0005829,cytosol,cellular_component
3,X6RLN4,GO:0005829,cytosol,cellular_component
4,X6RL45,GO:0016787,hydrolase activity,molecular_function
...,...,...,...,...
141480,A0A024CHX5,GO:0016787,hydrolase activity,molecular_function
141481,A0A024CHX5,GO:0005576,extracellular region,cellular_component
141482,A0A023T4H3,GO:0140096,"catalytic activity, acting on a protein",molecular_function
141483,A0A023IQH3,GO:0007155,cell adhesion,biological_process


In [170]:
df_quickgo['go_aspect'].value_counts()

go_aspect
cellular_component    53180
biological_process    46393
molecular_function    41912
Name: count, dtype: int64

In [171]:
df_quickgo['go_name'].value_counts()

go_name
nucleus                       9249
plasma membrane               6163
cytosol                       6135
transferase activity          5981
nucleoplasm                   5385
                              ... 
molecular sensor activity       37
respiratory system process      33
cytoplasmic translation         32
antioxidant activity            31
cyclase activity                30
Name: count, Length: 126, dtype: int64

In [172]:
for name, tokenizer in tqdm(list(tokenizer_list.items())):
    if 'pre' in name:
        df_protein_sliced[name] = [enc.tokens for enc in tokenizer.encode_batch(df_protein_sliced['sequence'])]
    else:
        df_protein[name] = [enc.tokens for enc in tokenizer.encode_batch(df_protein['sequence'])]

100%|██████████| 6/6 [00:13<00:00,  2.32s/it]


In [173]:
df_protein_sliced = df_protein_sliced.groupby('uniprot_id').sum().reset_index()
df_protein = df_protein.set_index(['uniprot_id', 'sequence']).join(df_protein_sliced.set_index(['uniprot_id', 'sequence'])).reset_index()
df_protein

,uniprot_id,sequence,PUMA blosum62 0.7 0.05 6400,PUMA blosum62 0.7 0.05 12800,PUMA blosum62 0.7 0.05 25600,PUMA pre pam70 0.7 0.05 6400,PUMA pre pam70 0.7 0.05 12800,PUMA pre pam70 0.7 0.05 25600
0,A0A087WZT3,MELSAEYLREKLQRDLEAEHVLPSPGGVGQVRGETAASETQLGS,"[MEL, SA, EYL, R, EKL, QR, DL, EA, EH, VL, PSP...","[MEL, SA, EYL, REKL, QRDL, EAEH, VL, PSP, GG, ...","[MEL, SA, EYL, REKL, QRDL, EAEH, VL, PSP, GGVG...","[MEL, SA, EYL, REKL, QRDL, EA, EH, VL, PSP, GG...","[MEL, SA, EYL, REKL, QRDL, EA, EHVL, PSP, GGVG...","[MEL, SA, EYL, REKL, QRDL, EA, EHVL, PSP, GGVG..."
1,A0A087X1C5,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,"[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,..."
2,A0A087X296,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,"[M, SR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, ...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, PG...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,..."
3,A0A0B4J2F0,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,"[M, FR, RL, TFA, QLL, FA, T, VLG, IA, GGV, YI,...","[M, FR, RL, TFA, QLL, FA, T, VLG, IA, GGV, YI,...","[M, FRRL, TFA, QLL, FAT, VLG, IA, GGV, YI, FQ,...","[M, FR, RL, TFA, QLL, FA, TVL, GIA, GGV, YI, F...","[M, FRRL, TFA, QLL, FA, TVL, GIA, GGV, YI, FQ,...","[M, FRRL, TFA, QLL, FA, TVL, GIA, GGV, YI, FQ,..."
4,A0A0C5B5G6,MRWQEMGYIFYPRKLR,"[M, RW, QE, MG, YI, FY, PR, KL, R]","[M, RW, QEMG, YI, FY, PRKL, R]","[MRW, QEMG, YI, FY, PRKL, R]","[M, RW, QE, MG, YI, FY, PR, KL, R]","[M, RW, QE, MG, YI, FY, PR, KL, R]","[M, RW, QE, MG, YI, FY, PRKL, R]"
...,...,...,...,...,...,...,...,...
70687,X6RL83,MLQEWLAAVGDDYAAVVWRPEGEPRFYPDEEGPKHWTKERHQFLME...,"[ML, QE, WL, AAVG, DDY, AA, VV, W, RP, EG, EPR...","[ML, QE, WL, AAVG, DDY, AA, VVW, RPEG, EPR, FY...","[ML, QE, WL, AAVG, DDY, AA, VVW, RPEG, EPR, FY...","[M, L, QE, WL, AAVG, DD, YAA, VV, W, RP, EG, E...","[M, L, QE, WL, AAVG, DD, YAA, VV, W, RP, EG, E...","[M, L, QE, WL, AAVG, DD, YAA, VV, W, RPEG, EPR..."
70688,X6RLN4,EVKGLFKSENCPKVISCEFAHNSNWYITFQSDTDAQQAFKYLREEV...,"[EV, KGL, FK, SEN, C, P, KVI, SC, EFA, HN, SN,...","[EV, KGL, FK, SEN, C, PKVI, SC, EFA, HN, SN, W...","[EV, KGL, FK, SEN, C, PKVI, SC, EFA, HN, SN, W...","[EV, KGL, FK, SEN, C, P, KVI, SC, EFA, HN, SNW...","[EV, KGL, FK, SEN, CP, KVI, SC, EFA, HN, SNW, ...","[EV, KGL, FK, SEN, CP, KVI, SC, EFA, HN, SNW, ..."
70689,X6RLR1,MAGLTDLQRLQARVEELERWVYGPGGARGSRKVADGLVKVQVALGN...,"[MA, GLT, DL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MA, GLT, DL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MA, GLT, DL, QRL, QARV, EEL, ER, WV, YG, PGG,...","[MA, GL, TDL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MAGL, TDL, QRL, QA, RV, EEL, ER, WV, YG, PGG,...","[MAGL, TDL, QRL, QARV, EEL, ER, WV, YG, PGG, A..."
70690,X6RLV5,MSGYSSDRDRGRDRGFGAPRFGGSRAGPLSGKKFGNPGEKLVKKKW...,"[MSG, YSS, DRD, RG, RD, RG, FGA, PRF, GG, SRAG...","[MSG, YSS, DRD, RG, RD, RG, FGA, PRF, GG, SRAG...","[MSG, YSS, DRD, RG, RDRG, FGA, PRF, GG, SRAG, ...","[MSG, Y, SSD, RD, RG, RD, RG, FGA, PR, FGG, SR...","[MSG, Y, SSD, RD, RG, RD, RG, FGA, PR, FGG, SR...","[MSG, Y, SSD, RD, RGRD, RG, FGA, PR, FGG, SRAG..."


In [174]:
df_protein_go = pd.merge(df_protein, df_quickgo, how='inner', on='uniprot_id')[['uniprot_id', 'go_id', 'go_name', 'go_aspect']+list(df_protein.columns[1:])].reset_index(drop=True)
df_protein_go

,uniprot_id,go_id,go_name,go_aspect,sequence,PUMA blosum62 0.7 0.05 6400,PUMA blosum62 0.7 0.05 12800,PUMA blosum62 0.7 0.05 25600,PUMA pre pam70 0.7 0.05 6400,PUMA pre pam70 0.7 0.05 12800,PUMA pre pam70 0.7 0.05 25600
0,A0A087X1C5,GO:0016491,oxidoreductase activity,molecular_function,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,"[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,..."
1,A0A087X1C5,GO:0005739,mitochondrion,cellular_component,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,"[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DLM, HRH, Q...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,...","[MGL, EALV, PL, AMI, VA, IF, LLLV, DL, MH, RH,..."
2,A0A087X296,GO:0006629,lipid metabolic process,biological_process,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,"[M, SR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, ...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, PG...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,..."
3,A0A087X296,GO:0016491,oxidoreductase activity,molecular_function,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,"[M, SR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, ...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, PG...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,..."
4,A0A087X296,GO:0005783,endoplasmic reticulum,cellular_component,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,"[M, SR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, ...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, WFLL, FLLLL, PPL, PVLL, AD, PGA, P...","[MSR, SLLL, W, FLL, FLL, LL, PPL, PVLL, AD, PG...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,...","[MSR, SLLL, WFLL, FLL, LL, PPL, PVLL, AD, PGA,..."
...,...,...,...,...,...,...,...,...,...,...,...
141480,X6RL45,GO:0016787,hydrolase activity,molecular_function,MVRCYVEIVEKLPERRPDPATIEGCAQLKPNNYLLAWHTPFNEKGS...,"[MV, RC, YV, EIV, EKL, PE, RR, PD, PA, TI, EG,...","[MV, RC, YV, EIV, EKL, PERR, PD, PA, TI, EG, C...","[MV, RC, YV, EIV, EKL, PERR, PD, PATI, EG, CA,...","[MV, RC, YV, EIV, EKL, PE, RR, PD, PA, TI, EG,...","[MV, RC, YV, EIV, EKL, PERR, PD, PA, TI, EG, C...","[MV, RC, YV, EIV, EKL, PERR, PDPA, TIEG, CA, Q..."
141481,X6RLN4,GO:0005829,cytosol,cellular_component,EVKGLFKSENCPKVISCEFAHNSNWYITFQSDTDAQQAFKYLREEV...,"[EV, KGL, FK, SEN, C, P, KVI, SC, EFA, HN, SN,...","[EV, KGL, FK, SEN, C, PKVI, SC, EFA, HN, SN, W...","[EV, KGL, FK, SEN, C, PKVI, SC, EFA, HN, SN, W...","[EV, KGL, FK, SEN, C, P, KVI, SC, EFA, HN, SNW...","[EV, KGL, FK, SEN, CP, KVI, SC, EFA, HN, SNW, ...","[EV, KGL, FK, SEN, CP, KVI, SC, EFA, HN, SNW, ..."
141482,X6RLR1,GO:0000910,cytokinesis,biological_process,MAGLTDLQRLQARVEELERWVYGPGGARGSRKVADGLVKVQVALGN...,"[MA, GLT, DL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MA, GLT, DL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MA, GLT, DL, QRL, QARV, EEL, ER, WV, YG, PGG,...","[MA, GL, TDL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MAGL, TDL, QRL, QA, RV, EEL, ER, WV, YG, PGG,...","[MAGL, TDL, QRL, QARV, EEL, ER, WV, YG, PGG, A..."
141483,X6RLR1,GO:0005730,nucleolus,cellular_component,MAGLTDLQRLQARVEELERWVYGPGGARGSRKVADGLVKVQVALGN...,"[MA, GLT, DL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MA, GLT, DL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MA, GLT, DL, QRL, QARV, EEL, ER, WV, YG, PGG,...","[MA, GL, TDL, QRL, QA, RV, EEL, ER, WV, YG, PG...","[MAGL, TDL, QRL, QA, RV, EEL,

In [175]:
# Protein unitlerini "doküman" formatına dönüştürme
def create_unit_documents(df: pd.DataFrame, tokenizer_col: str, token_len_thr: int = 0) -> List[str]:
    """Her protein için tokenizer çıktılarını boşluklarla ayrılmış doküman formatına dönüştürür"""
    # return df[tokenizer_col].apply(lambda units: ' '.join(units)).tolist()
    return df[tokenizer_col].apply(lambda units: ' '.join(unit for unit in units if len(unit) >= token_len_thr)).tolist()

# GO terimlerini etiketlere dönüştürme
def create_go_labels(df: pd.DataFrame, go_col: str = 'go_id') -> List[List[str]]:
    """Her protein için GO terimlerini liste formatında döndürür"""
    return df[go_col].tolist()

# BERTopic modeli oluşturma
def create_bertopic_model(documents: List[str], go_labels: List[str], token_len_thr: int = 0) -> Tuple[BERTopic, np.ndarray]:
    """Manuel topic modelleme için BERTopic modeli oluşturur"""
    
    # GO terimleri için one-hot encoding
    lb = LabelBinarizer()
    go_binary = lb.fit_transform(go_labels)
    
    # CountVectorizer ayarları - protein unit'ler için
    vectorizer_model = CountVectorizer(
        lowercase=False,
        token_pattern=r"(?u)\b\w{%d,}\b" %token_len_thr,
        stop_words=None,  # Protein unit'lerde stop word kullanmıyoruz
        ngram_range=(1, 1),  # Tek ve çift protein unit'leri de değerlendirelim
        min_df=5,  # En az 5 proteinde görünen unit'leri alalım
        max_df=0.7,  # Proteinlerin %70'inden fazlasında görünenleri çıkaralım
    )
    
    # Protein unit'lerin önemi için özel TF-IDF
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
    
    # Terimlerin temsil edilmesi için model
    representation_model = KeyBERTInspired()
    
    empty_embedding_model = BaseEmbedder()
    empty_dimensionality_model = BaseDimensionalityReduction()
    empty_cluster_model = BaseCluster()

    # BERTopic model oluşturma - embedding modeli belirtmiyoruz (manuel topic modelleme)
    topic_model = BERTopic(
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        embedding_model=empty_embedding_model,
        umap_model=empty_dimensionality_model,
        hdbscan_model=empty_cluster_model,
        # representation_model=representation_model,
        # min_topic_size=10,  # En az 10 protein içeren topic'leri kabul edelim
        # calculate_probabilities=True,
        verbose=True
    )
    
    # Topic'leri manuel olarak GO terimleriyle eşleştir
    topics = np.argmax(go_binary, axis=1)
    
    # Eğer bir protein için hiçbir GO terimi yoksa, -1 (outlier) olarak işaretle
    topics[np.sum(go_binary, axis=1) == 0] = -1
    
    # Topic isimlerini go_slim_ids değerleriyle eşleştir
    topic_labels = {i: lb.classes_[i] for i in range(len(lb.classes_))}
    topic_labels[-1] = "Outlier"  # Outlier topic'i için etiket
    
    # Belgeleri ve topic'leri kullanarak modeli eğit
    topic_model.fit_transform(documents, y=topics)
    topic_model.set_topic_labels(topic_labels)
    
    return topic_model, topics

In [176]:
def compute_smoothing_matrix(similarity_matrix, lambda_smooth=0.1):
        """Precompute the smoothing matrix (I + λA)"""
        # Normalize adjacency matrix (row-wise normalization)
        if similarity_matrix is None:
            return None
        
        A = similarity_matrix.copy()
        row_sums = np.array(A.sum(axis=1)).flatten()
        row_sums[row_sums == 0] = 1  # Avoid division by zero
        A_norm = A / row_sums[:, np.newaxis]
        
        # Create smoothing matrix: (I + λA)
        I = np.eye(A_norm.shape[0])
        smoothing_matrix = I + lambda_smooth * A_norm
        
        return smoothing_matrix

def apply_graph_smoothing(X, smoothing_matrix):
    """
    Apply graph smoothing to document-term matrix before c-TF-IDF
    
    Args:
        X: Document-term matrix (n_documents x n_features)
        similarity_matrix: Unit similarity matrix (n_features x n_features)
        lambda_smooth: Smoothing parameter
        
    Returns:
        Smoothed document-term matrix
    """
    if smoothing_matrix is None:
        return X
    
    # Apply smoothing: D' = D * (I + λA)
    if sp.issparse(X):
        X_smoothed = X.dot(sp.csr_matrix(smoothing_matrix))
    else:
        X_smoothed = np.dot(X, smoothing_matrix)
        
    return X_smoothed


class GraphAwareCountVectorizer(CountVectorizer):
    """CountVectorizer that applies graph smoothing to the resulting matrix"""
    
    def __init__(self, 
                 similarity_matrix=None,
                 lambda_smooth=0.1,
                 **kwargs):
        super().__init__(**kwargs)
        self.lambda_smooth = lambda_smooth
        self.similarity_matrix = similarity_matrix
        self.smoothing_matrix = None
    
    def fit_transform(self, raw_documents, y=None):
        """Fit and transform with graph smoothing"""
        X = super().fit_transform(raw_documents, y)

        self.smoothing_matrix = compute_smoothing_matrix(self.similarity_matrix, self.lambda_smooth)
        
        # Apply graph smoothing if similarity matrix is available
        if self.smoothing_matrix is not None:
            # print('fit_transform', X.shape, self.smoothing_matrix.shape)
            X = apply_graph_smoothing(X, self.smoothing_matrix)
            
        return X
    
    def transform(self, raw_documents):
        """Transform with graph smoothing"""
        X = super().transform(raw_documents)
        # print('transform', X.shape, self.smoothing_matrix.shape)
        
        # Apply graph smoothing if similarity matrix is available
        if self.smoothing_matrix is not None:
            X = apply_graph_smoothing(X, self.smoothing_matrix)
            
        return X



def build_unit_similarity_matrix(unit_relationships: Dict[str, Dict], 
                                vocabulary: List[str],
                                alpha: float = 1.0, 
                                beta: float = 0.5,
                                theta: float = 0.7) -> np.ndarray:
    """
    Build adjacency matrix for protein unit relationships
    
    Args:
        unit_relationships: Dictionary containing unit relationships
            Expected format:
            {
                'hierarchical': {
                    'parent_unit': ['child1', 'child2', ...],
                    ...
                },
                'mutational': {
                    'parent1': ['child1', 'child2', ...],
                    'parent2': ['child1', 'child2', ...],
                    ...
                }
            }
        vocabulary: List of all protein units (vocabulary from CountVectorizer)
        alpha: Weight for merge parent-child relationships
        beta: Weight for sibling relationships
        theta: Weight for mutation parent-child relationships
        
    Returns:
        Adjacency matrix (n_units x n_units)
    """
    
    # Create unit to index mapping
    unit_to_idx = {unit: idx for idx, unit in enumerate(vocabulary)}
    n_units = len(vocabulary)
    
    # Initialize adjacency matrix
    A = np.zeros((n_units, n_units))
    
    # Add hierarchical relationships (parent-child)
    if 'hierarchical' in unit_relationships:
        for parent, children in unit_relationships['hierarchical'].items():
            if parent in unit_to_idx:
                parent_idx = unit_to_idx[parent]
                for child in children:
                    if child in unit_to_idx:
                        child_idx = unit_to_idx[child]
                        # Bidirectional parent-child relationship
                        A[parent_idx, child_idx] = alpha
                        A[child_idx, parent_idx] = alpha
    
    # Add mutational relationships (family-based)
    if 'mutational' in unit_relationships:
        for parent, children in unit_relationships['mutational'].items():
            if parent in unit_to_idx:
                parent_idx = unit_to_idx[parent]
                for i in range(len(children)):
                    if children[i] in unit_to_idx:
                        child_i_idx = unit_to_idx[children[i]]
                        # Parent-child relationship
                        A[parent_idx, child_i_idx] = theta
                        A[child_i_idx, parent_idx] = theta
                        for j in range(i+1, len(children)):
                            if children[j] in unit_to_idx:
                                child_j_idx = unit_to_idx[children[j]]
                                # Sibling relationship
                                A[child_i_idx, child_j_idx] = beta
                                A[child_j_idx, child_i_idx] = beta
    
    return A


def create_graph_aware_bertopic_model(documents: List[str], 
                                    go_labels: List[str],
                                    unit_relationships: Optional[Dict[str, Dict]] = None,
                                    token_len_thr: int = 0,
                                    lambda_smooth: float = 0.1,
                                    alpha: float = 1.0,
                                    beta: float = 0.5,
                                    theta: float = 0.7) -> Tuple[BERTopic, np.ndarray, np.ndarray]:
    """
    Create Graph-Aware BERTopic model for protein analysis
    
    Args:
        documents: List of protein sequences (tokenized with protein units)
        go_labels: List of GO-slim labels for each protein
        unit_relationships: Dictionary containing unit relationships (hierarchical & mutational)
        token_len_thr: Minimum token length threshold
        lambda_smooth: Graph smoothing parameter
        alpha: Weight for parent-child relationships
        beta: Weight for sibling relationships
        theta: Weight for mutation parent-child relationships
        
    Returns:
        Tuple of (BERTopic model, topics array, similarity matrix)
    """
    
    # GO terms için one-hot encoding
    lb = LabelBinarizer()
    go_binary = lb.fit_transform(go_labels)
    
    # CountVectorizer ayarları - protein units için
    temp_vectorizer = CountVectorizer(
        lowercase=False,
        token_pattern=r"(?u)\b\w{%d,}\b" % token_len_thr,
        stop_words=None,
        ngram_range=(1, 1),
        min_df=5,
        max_df=0.7,
    )
    
    temp_vectorizer.fit(documents)
    vocabulary = temp_vectorizer.get_feature_names_out().tolist()

    # Build unit similarity matrix
    similarity_matrix = None
    if unit_relationships is not None:
        similarity_matrix = build_unit_similarity_matrix(
            unit_relationships=unit_relationships,
            vocabulary=vocabulary,
            alpha=alpha,
            beta=beta,
            theta=theta
        )
        print(f"Built similarity matrix with shape: {similarity_matrix.shape}")
        print(f"Non-zero edges: {np.sum(similarity_matrix > 0)}")
    
    # Create graph-aware CountVectorizer
    vectorizer_model = GraphAwareCountVectorizer(
        similarity_matrix=similarity_matrix,
        lambda_smooth=lambda_smooth,
        lowercase=False,
        token_pattern=r"(?u)\b\w{%d,}\b" % token_len_thr,
        stop_words=None,
        ngram_range=(1, 1),
        min_df=5,
        max_df=0.7,
        vocabulary=vocabulary  # Use the same vocabulary
    )
    
    # Standard c-TF-IDF model (no need for custom one now)
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
    
    # Representation model
    representation_model = KeyBERTInspired()
    
    # Empty models for manual topic assignment
    empty_embedding_model = BaseEmbedder()
    empty_dimensionality_model = BaseDimensionalityReduction()
    empty_cluster_model = BaseCluster()
    
    # BERTopic model with graph-aware c-TF-IDF
    topic_model = BERTopic(
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        embedding_model=empty_embedding_model,
        umap_model=empty_dimensionality_model,
        hdbscan_model=empty_cluster_model,
        # representation_model=representation_model,
        # min_topic_size=10,  # En az 10 protein içeren topic'leri kabul edelim
        # calculate_probabilities=True,
        verbose=True
    )
    
    # Manual topic assignment based on GO terms
    topics = np.argmax(go_binary, axis=1)
    
    # Mark proteins without GO terms as outliers
    topics[np.sum(go_binary, axis=1) == 0] = -1
    
    # Create topic labels
    topic_labels = {i: lb.classes_[i] for i in range(len(lb.classes_))}
    topic_labels[-1] = "Outlier"
    
    # Train the model
    topic_model.fit_transform(documents, y=topics)
    topic_model.set_topic_labels(topic_labels)
    
    print(f"Model trained with {len(set(topics))} topics")
    print(f"Graph smoothing parameter λ = {lambda_smooth}")
    
    return topic_model, topics, similarity_matrix



In [177]:
def split_data(df):
    # Tüm benzersiz UniProt ID'leri al
    unique_uniprot_ids = df["uniprot_id"].unique().tolist()
    random.shuffle(unique_uniprot_ids)

    # ID'lerin %80-%20 şeklinde split edilmesi
    train_ids, test_ids = train_test_split(unique_uniprot_ids, test_size=0.2, random_state=42)

    # Verileri splitlere ayır
    df_train = df[df["uniprot_id"].isin(train_ids)].copy()
    df_test = df[df["uniprot_id"].isin(test_ids)].copy()

    # GO dağılımlarını kontrol et
    def go_dist(data):
        return data['go_id'].value_counts(normalize=True)

    train_dist = go_dist(df_train)
    test_dist = go_dist(df_test)

    # Dağılım farklarını görmek için karşılaştıralım
    distribution_df = pd.DataFrame({
        "train_ratio": train_dist,
        "test_ratio": test_dist
    }).fillna(0)

    distribution_df["difference"] = abs(distribution_df["train_ratio"] - distribution_df["test_ratio"])

    # Gerekirse tolerans üstü farkları göster
    print(distribution_df.sort_values("difference", ascending=False).head(5))

    return df_train.reset_index(drop=True), df_test.reset_index(drop=True)


In [178]:
def evaluate_go_term_representations(df_test: pd.DataFrame, topic_model: BERTopic, tokenizer_col: str, go_col: str = 'go_name') -> Dict[str, Any]:
    """
    Evaluates the learned GO term representations on the test set using various metrics.

    Args:
        df_test: The test dataframe containing protein sequences and their true GO terms.
        topic_model: The trained BERTopic model.
        tokenizer_col: The name of the column in df_test with the tokenized sequences.
        go_col: The name of the column in df_test with the true GO term labels.

    Returns:
        A dictionary containing the evaluation results.
    """
    
    # --- Step 1: Pre-process topic representations for faster lookup ---
    # Create a mapping from GO name to a list of its representative (unit, score) tuples.
    go_term_representations = {}
    topic_info = topic_model.get_topic_info()
    
    # BERTopic might have a different internal topic ID than the label index.
    # We need to map the topic ID to the GO name (custom label).
    topic_id_to_go_name = {row.Topic: row.CustomName for _, row in topic_info.iterrows()}

    for topic_id, go_name in topic_id_to_go_name.items():
        if topic_id == -1:  # Skip outlier topic
            continue
        
        # Get the top 10 representative units and their scores for the current topic
        representation = topic_model.get_topic(topic_id)
        if representation:
            go_term_representations[go_name] = {unit: score for unit, score in representation}

    # --- Step 2: Iterate through the test set and evaluate ---
    y_true = []
    y_pred = []
    reciprocal_ranks = []
    
    # Group by protein to handle multiple GO terms per protein if necessary,
    # though the current setup seems to be one primary GO term per row.
    test_proteins = df_test.groupby('uniprot_id')

    for uniprot_id, protein_data in tqdm(test_proteins, desc="Evaluating Test Set"):
        # Use the first row's data for simplicity, assuming one main GO term per protein.
        true_go_name = protein_data.iloc[0][go_col]
        
        # The tokenized sequence for this protein. Use a set for O(1) lookups.
        tokenized_sequence = set(protein_data.iloc[0][tokenizer_col])
        
        if not tokenized_sequence:
            continue

        # --- Step 3: Score all possible GO terms for the current protein ---
        go_scores = {}
        for go_name, representation in go_term_representations.items():
            score = 0
            # Sum the scores of representative units that are present in the protein's sequence
            for unit, unit_score in representation.items():
                if unit in tokenized_sequence:
                    score += unit_score
            go_scores[go_name] = score
            
        if not go_scores:
            continue

        # Sort GO terms by their calculated score in descending order
        sorted_go_terms = sorted(go_scores.items(), key=lambda item: item[1], reverse=True)
        
        # The top prediction is the GO term with the highest score
        predicted_go_name = sorted_go_terms[0][0]
        
        y_true.append(true_go_name)
        y_pred.append(predicted_go_name)
        
        # --- Step 4: Calculate rank of the true GO term for MRR ---
        rank = -1
        for i, (go_name, score) in enumerate(sorted_go_terms):
            if go_name == true_go_name:
                rank = i + 1
                break
        
        if rank != -1:
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0) # True GO term was not found in the ranked list

    # --- Step 5: Calculate and compile final metrics ---
    results = {}

    # --- Metric: Top-k Accuracy ---
    # Calculates the percentage of times the true GO term is in the top k predictions.
    top_k_accuracies = {}
    for k in [1, 3, 5, 10]:
        correct_predictions = 0
        for i in range(len(y_true)):
            # Re-calculate sorted list for each protein to check top-k
            # This is slightly redundant but clear. Could be optimized by storing all sorted lists.
            protein_data = df_test[df_test['uniprot_id'] == df_test['uniprot_id'].unique()[i]]
            tokenized_sequence = set(protein_data.iloc[0][tokenizer_col])
            go_scores = {go: sum(score for unit, score in go_term_representations.get(go, {}).items() if unit in tokenized_sequence) for go in go_term_representations}
            sorted_go_terms = [item[0] for item in sorted(go_scores.items(), key=lambda item: item[1], reverse=True)]
            
            if y_true[i] in sorted_go_terms[:k]:
                correct_predictions += 1
        top_k_accuracies[f'accuracy_at_{k}'] = correct_predictions / len(y_true) if y_true else 0
    
    results['top_k_accuracy'] = top_k_accuracies
    
    # --- Metric: Mean Reciprocal Rank (MRR) ---
    # The average of the reciprocal ranks of the correct GO term.
    # A value of 1 means the correct answer is always the first prediction.
    # A value of 0.5 means the correct answer is, on average, the second prediction.
    results['mean_reciprocal_rank'] = np.mean(reciprocal_ranks) if reciprocal_ranks else 0

    # --- Metric: Classification Report (Precision, Recall, F1-Score) ---
    # Based on the top-1 prediction (the GO term with the highest weighted score).
    # Precision: Of all proteins predicted to have a certain GO term, how many were correct?
    # Recall: Of all proteins that truly have a certain GO term, how many did we identify?
    # F1-Score: The harmonic mean of Precision and Recall.
    # Accuracy: Overall, what fraction of predictions were correct?
    if y_true and y_pred:
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
        results['classification_report'] = {
            'accuracy': report['accuracy'],
            'macro_avg': report['macro avg'],
            'weighted_avg': report['weighted avg']
        }

    return results


In [186]:
def evaluate_go_term_representations(df_test: pd.DataFrame, topic_model: BERTopic, tokenizer_col: str, go_col: str = 'go_name') -> Dict[str, Any]:
    """
    Evaluates the learned GO term representations on the test set using various metrics.
    This version is optimized for speed and includes per-GO-term metrics and frequency-based scoring.

    Args:
        df_test: The test dataframe containing protein sequences and their true GO terms.
        topic_model: The trained BERTopic model.
        tokenizer_col: The name of the column in df_test with the tokenized sequences.
        go_col: The name of the column in df_test with the true GO term labels.

    Returns:
        A dictionary containing the evaluation results.
    """
    
    # --- Step 1: Pre-process topic representations for faster lookup ---
    go_term_representations = {}
    topic_info = topic_model.get_topic_info()
    topic_id_to_go_name = {row.Topic: row.CustomName for _, row in topic_info.iterrows()}

    for topic_id, go_name in topic_id_to_go_name.items():
        if topic_id == -1:
            continue
        representation = topic_model.get_topic(topic_id)
        if representation:
            go_term_representations[go_name] = {unit: score for unit, score in representation}

    # --- Step 2: Iterate through the test set to get predictions and scores ---
    y_true = []
    all_sorted_predictions = []
    reciprocal_ranks = []
    
    test_proteins = df_test.groupby('uniprot_id')

    for uniprot_id, protein_data in tqdm(test_proteins, desc="Evaluating Test Set"):
        true_go_name = protein_data.iloc[0][go_col]
        
        # Use a Counter to get frequencies of each unit for the new scoring metric.
        tokenized_sequence = protein_data.iloc[0][tokenizer_col]
        if not tokenized_sequence:
            continue
        token_counts = Counter(tokenized_sequence)

        # --- Step 3: Score all possible GO terms for the current protein ---
        go_scores = {}
        for go_name, representation in go_term_representations.items():
            # Sum the scores of representative units that are present in the protein's sequence
            score = sum(unit_score for unit, unit_score in representation.items() if unit in tokenized_sequence)
            # New Metric: Score is the sum of (unit_score * unit_frequency_in_protein)
            # score = sum(unit_score * token_counts.get(unit, 0) for unit, unit_score in representation.items())
            
            go_scores[go_name] = score
            
        if not go_scores:
            continue

        # Sort GO terms by their calculated score in descending order
        sorted_go_terms = sorted(go_scores.items(), key=lambda item: item[1], reverse=True)
        
        # Store the full sorted list of predictions for efficient top-k calculation later
        sorted_prediction_names = [go_name for go_name, score in sorted_go_terms]
        all_sorted_predictions.append(sorted_prediction_names)
        y_true.append(true_go_name)
        
        # --- Step 4: Calculate rank of the true GO term for MRR ---
        try:
            rank = sorted_prediction_names.index(true_go_name) + 1
            reciprocal_ranks.append(1.0 / rank)
        except ValueError:
            # The true GO term was not in the list of predictions
            reciprocal_ranks.append(0.0)

    # --- Step 5: Calculate and compile final metrics ---
    results = {}
    
    if not y_true:
        return {"error": "No test data to evaluate."}

    # --- Metric: Top-k Accuracy (Optimized) ---
    # Calculates the percentage of times the true GO term is in the top k predictions.
    top_k_accuracies = {}
    for k in [1, 3, 5, 10]:
        correct_predictions = sum(1 for i, true_label in enumerate(y_true) if true_label in all_sorted_predictions[i][:k])
        top_k_accuracies[f'accuracy_at_{k}'] = correct_predictions / len(y_true)
    
    results['top_k_accuracy'] = top_k_accuracies
    
    # --- Metric: Mean Reciprocal Rank (MRR) ---
    # The average of the reciprocal ranks of the correct GO term.
    results['mean_reciprocal_rank'] = np.mean(reciprocal_ranks) if reciprocal_ranks else 0

    # --- Metric: Classification Report (Precision, Recall, F1-Score) ---
    # Based on the top-1 prediction (the GO term with the highest weighted score).
    y_pred = [preds[0] if preds else None for preds in all_sorted_predictions]
    
    # Filter out None predictions if any protein had no scoreable GO terms
    valid_indices = [i for i, p in enumerate(y_pred) if p is not None]
    y_true_filtered = [y_true[i] for i in valid_indices]
    y_pred_filtered = [y_pred[i] for i in valid_indices]

    if y_true_filtered and y_pred_filtered:
        # Get all labels present in either true or predicted sets for the report
        labels = sorted(list(set(y_true_filtered) | set(y_pred_filtered)))
        report = classification_report(y_true_filtered, y_pred_filtered, labels=labels, output_dict=True, zero_division=0)
        
        # Separate overall metrics from per-GO-term metrics
        results['classification_summary'] = {
            'accuracy': report['accuracy'],
            'macro_avg': report['macro avg'],
            'weighted_avg': report['weighted avg']
        }
        
        # --- New: Per-GO-Term Metrics ---
        # Detailed precision, recall, f1-score for each individual GO term.
        per_go_term_metrics = {label: report[label] for label in labels if label in report}
        results['per_go_term_metrics'] = per_go_term_metrics

    return results

In [200]:
def evaluate_go_term_representations_multilabel(df_test: pd.DataFrame, topic_model: BERTopic, tokenizer_col: str, go_col: str = 'go_name') -> Dict[str, Any]:
    """
    Evaluates GO term representations considering all GO terms for each protein (multi-label evaluation).
    Includes multi-label accuracy@k, precision@k, recall@k, and a full classification report.

    Args:
        df_test: The test dataframe with all GO term associations for each protein.
        topic_model: The trained BERTopic model.
        tokenizer_col: The name of the column with tokenized sequences.
        go_col: The name of the column with GO term labels.

    Returns:
        A dictionary with comprehensive multi-label evaluation results.
    """
    
    # --- Step 1: Pre-process topic representations ---
    go_term_representations = {}
    topic_info = topic_model.get_topic_info()
    topic_id_to_go_name = {row.Topic: row.CustomName for _, row in topic_info.iterrows()}

    for topic_id, go_name in topic_id_to_go_name.items():
        if topic_id == -1:
            continue
        representation = topic_model.get_topic(topic_id)
        if representation:
            go_term_representations[go_name] = {unit: score for unit, score in representation}

    # --- Step 2: Get predictions for each protein in the test set ---
    y_true_sets = []
    all_sorted_predictions = []
    
    test_proteins = df_test.groupby('uniprot_id')

    for uniprot_id, protein_data in tqdm(test_proteins, desc="Evaluating Test Set (Multi-Label)"):
        true_go_names = set(protein_data[go_col].tolist())
        tokenized_sequence = protein_data.iloc[0][tokenizer_col]
        if not tokenized_sequence:
            continue
        token_counts = Counter(tokenized_sequence)

        # Sum the scores of representative units that are present in the protein's sequence
        go_scores = {go_name: sum(unit_score for unit, unit_score in representation.items() if unit in tokenized_sequence) for go_name, representation in go_term_representations.items()}
        # New Metric: Score is the sum of (unit_score * unit_frequency_in_protein)
        # go_scores = {go_name: sum(unit_score * token_counts.get(unit, 0) for unit, unit_score in representation.items()) for go_name, representation in go_term_representations.items()}
        
        if not go_scores:
            continue

        sorted_prediction_names = [go_name for go_name, score in sorted(go_scores.items(), key=lambda item: item[1], reverse=True)]
        
        all_sorted_predictions.append(sorted_prediction_names)
        y_true_sets.append(true_go_names)

    # --- Step 3: Calculate and compile multi-label metrics ---
    results = {}
    if not y_true_sets:
        return {"error": "No test data to evaluate."}

    # --- Metric: Mean Reciprocal Rank (MRR) ---
    reciprocal_ranks = []
    for i, true_set in enumerate(y_true_sets):
        rank = next((r + 1 for r, p in enumerate(all_sorted_predictions[i]) if p in true_set), 0)
        reciprocal_ranks.append(1.0 / rank if rank > 0 else 0.0)
    results['mean_reciprocal_rank'] = np.mean(reciprocal_ranks) if reciprocal_ranks else 0

    # --- Metrics: Accuracy@k, Precision@k, and Recall@k ---
    accuracy_at_k = {}
    precision_at_k = {}
    recall_at_k = {}
    
    for k in [1, 3, 5, 10]:
        acc_at_k_scores = []
        p_at_k_scores = []
        r_at_k_scores = []
        for i, true_set in enumerate(y_true_sets):
            if not true_set: continue
            
            pred_set_at_k = set(all_sorted_predictions[i][:k])
            hits = len(true_set.intersection(pred_set_at_k))
            
            # Accuracy@k: 1 if there is at least one hit, 0 otherwise.
            acc_at_k_scores.append(1.0 if hits > 0 else 0.0)
            p_at_k_scores.append(hits / k)
            r_at_k_scores.append(hits / len(true_set))
            
        accuracy_at_k[f'accuracy_at_{k}'] = np.mean(acc_at_k_scores) if acc_at_k_scores else 0
        precision_at_k[f'precision_at_{k}'] = np.mean(p_at_k_scores) if p_at_k_scores else 0
        recall_at_k[f'recall_at_{k}'] = np.mean(r_at_k_scores) if r_at_k_scores else 0

    results['accuracy_at_k'] = accuracy_at_k
    results['precision_at_k'] = precision_at_k
    results['recall_at_k'] = recall_at_k

    # --- Metric: Multi-Label Classification Report ---
    # To create a report, we must convert the ranking into a binary prediction.
    # Strategy: For each sample, predict the top N labels, where N is the number of true labels for that sample.
    all_labels = sorted(list(go_term_representations.keys()))
    mlb = MultiLabelBinarizer(classes=all_labels)
    
    y_true_binary = mlb.fit_transform(y_true_sets)
    y_pred_binary = []

    for i, true_set in enumerate(y_true_sets):
        # Predict top k labels where k is the number of true labels
        k = len(true_set)
        if k == 0: continue
        top_k_preds = all_sorted_predictions[i][:k]
        y_pred_binary.append(mlb.transform([top_k_preds])[0])

    if y_pred_binary:
        y_pred_binary = np.array(y_pred_binary)
        # We need to align y_true_binary with the proteins we actually made predictions for
        y_true_binary_filtered = [y_true_binary[i] for i, ts in enumerate(y_true_sets) if len(ts) > 0]

        report = classification_report(
            y_true_binary_filtered, 
            y_pred_binary, 
            target_names=all_labels, 
            output_dict=True, 
            zero_division=0
        )
        
        results['classification_summary'] = {
            'micro_avg': report['micro avg'],
            'macro_avg': report['macro avg'],
            'weighted_avg': report['weighted avg'],
            'samples_avg': report['samples avg']
        }
        
        per_go_term_metrics = {label: report[label] for label in all_labels if label in report}
        results['per_go_term_metrics'] = per_go_term_metrics

    return results

In [179]:
# df = df_protein_go.copy()
df = df_protein_go[df_protein_go['go_aspect'] == 'cellular_component'].reset_index(drop=True).copy()
df_train, df_test = split_data(df)

tokenizer_col = 'PUMA blosum62 0.7 0.05 12800'
go_col = 'go_name'
token_len_thr = 0

unit_relationships = {'hierarchical': {}, 'mutational': {}}
for unit, lineage in vocab_lineage_list[tokenizer_col].items():
    if len(lineage['child_pair']) > 0:
        unit_relationships['hierarchical'][unit] = lineage['child_pair']
    if len(lineage['child_mutation']) > 0:
        unit_relationships['mutational'][unit] = lineage['child_mutation']

print(f"Tokenizer kullanılıyor: {tokenizer_col}")
documents = create_unit_documents(df_train, tokenizer_col, token_len_thr)
go_labels = create_go_labels(df_train, go_col)

            train_ratio  test_ratio  difference
go_id                                          
GO:0005829     0.114518    0.118779    0.004261
GO:0005576     0.071755    0.074699    0.002944
GO:0005634     0.173395    0.176036    0.002641
GO:0005654     0.101733    0.099346    0.002388
GO:0005929     0.018766    0.017063    0.001702
Tokenizer kullanılıyor: PUMA blosum62 0.7 0.05 12800


In [180]:
print("BERTopic modeli oluşturuluyor...")
topic_model, topics = create_bertopic_model(documents, go_labels, token_len_thr=4)

2025-07-01 00:17:55,908 - BERTopic - Embedding - Transforming documents to embeddings.
2025-07-01 00:17:55,909 - BERTopic - Embedding - Completed ✓
2025-07-01 00:17:55,910 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-01 00:17:55,910 - BERTopic - Dimensionality - Completed ✓
2025-07-01 00:17:55,911 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-01 00:17:55,913 - BERTopic - Cluster - Completed ✓
2025-07-01 00:17:55,919 - BERTopic - Representation - Fine-tuning topics using representation models.


BERTopic modeli oluşturuluyor...


2025-07-01 00:17:56,559 - BERTopic - Representation - Completed ✓


In [203]:
# print("BERTopic modeli oluşturuluyor...")
topic_model, topics, similarity_matrix = create_graph_aware_bertopic_model(documents, go_labels, unit_relationships, token_len_thr=4, lambda_smooth=0.7, alpha=1, beta=0.5, theta=0.7)

2025-07-01 01:09:53,532 - BERTopic - Embedding - Transforming documents to embeddings.
2025-07-01 01:09:53,534 - BERTopic - Embedding - Completed ✓
2025-07-01 01:09:53,534 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-01 01:09:53,534 - BERTopic - Dimensionality - Completed ✓
2025-07-01 01:09:53,535 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-01 01:09:53,538 - BERTopic - Cluster - Completed ✓
2025-07-01 01:09:53,543 - BERTopic - Representation - Fine-tuning topics using representation models.


Built similarity matrix with shape: (8155, 8155)
Non-zero edges: 191212


2025-07-01 01:09:54,720 - BERTopic - Representation - Completed ✓


Model trained with 25 topics
Graph smoothing parameter λ = 0.7


In [204]:
# --- EVALUATION EXAMPLE ---
# Now, call the evaluation function with the test data and the trained model
evaluation_results = evaluate_go_term_representations(df_test, topic_model, tokenizer_col, go_col)

# Print the results in a readable format
import json
print("\n--- GO Term Representation Evaluation Results ---")
print(json.dumps(evaluation_results, indent=2))

Evaluating Test Set: 100%|██████████| 5886/5886 [00:01<00:00, 3200.75it/s]



--- GO Term Representation Evaluation Results ---
{
  "top_k_accuracy": {
    "accuracy_at_1": 0.03856608902480462,
    "accuracy_at_3": 0.08341828066598708,
    "accuracy_at_5": 0.13302752293577982,
    "accuracy_at_10": 0.36493374108053006
  },
  "mean_reciprocal_rank": 0.13190339625620615,
  "classification_summary": {
    "accuracy": 0.03856608902480462,
    "macro_avg": {
      "precision": 0.04505983249060523,
      "recall": 0.05537094435395813,
      "f1-score": 0.025695922545264496,
      "support": 5886.0
    },
    "weighted_avg": {
      "precision": 0.14757174649226928,
      "recall": 0.03856608902480462,
      "f1-score": 0.039826980239771714,
      "support": 5886.0
    }
  },
  "per_go_term_metrics": {
    "Golgi apparatus": {
      "precision": 0.027186761229314422,
      "recall": 0.39655172413793105,
      "f1-score": 0.05088495575221239,
      "support": 174.0
    },
    "chromosome": {
      "precision": 0.0,
      "recall": 0.0,
      "f1-score": 0.0,
      "sup

In [205]:
# --- MULTI-LABEL EVALUATION EXAMPLE ---
# Call the new multi-label evaluation function
multilabel_evaluation_results = evaluate_go_term_representations_multilabel(df_test, topic_model, tokenizer_col, go_col)

print("\n--- GO Term Representation Evaluation Results (Multi-Label) ---")
print(json.dumps(multilabel_evaluation_results, indent=2))

Evaluating Test Set (Multi-Label):   0%|          | 0/5886 [00:00<?, ?it/s]

Evaluating Test Set (Multi-Label): 100%|██████████| 5886/5886 [00:04<00:00, 1268.87it/s]



--- GO Term Representation Evaluation Results (Multi-Label) ---
{
  "mean_reciprocal_rank": 0.19873226921698717,
  "accuracy_at_k": {
    "accuracy_at_1": 0.07509344206591913,
    "accuracy_at_3": 0.17040434930343187,
    "accuracy_at_5": 0.25925925925925924,
    "accuracy_at_10": 0.5524974515800204
  },
  "precision_at_k": {
    "precision_at_1": 0.07509344206591913,
    "precision_at_3": 0.06082229018008835,
    "precision_at_5": 0.0600067957866123,
    "precision_at_10": 0.07456676860346585
  },
  "recall_at_k": {
    "recall_at_1": 0.04033744316618934,
    "recall_at_3": 0.09345042581535955,
    "recall_at_5": 0.15110033763193803,
    "recall_at_10": 0.3924972628081701
  },
  "classification_summary": {
    "micro_avg": {
      "precision": 0.10294814674376718,
      "recall": 0.10294814674376718,
      "f1-score": 0.10294814674376718,
      "support": 10549.0
    },
    "macro_avg": {
      "precision": 0.11357729450920574,
      "recall": 0.12520409218460718,
      "f1-score": 0